### UNet Calgary

In [4]:
import os
import collections
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
import sys
import wandb
from batchgenerators.dataloading.multi_threaded_augmenter import MultiThreadedAugmenter
from batchgenerators.transforms.abstract_transforms import Compose

sys.path.append('../../')
from utils import volume_collate
from dataset import CalgaryCampinasDataset
from data_utils import (
    Transforms,
    MultiImageSingleViewDataLoader
)
from model.unet import UNet2D
from losses import (
    DiceScoreCalgary, 
    SurfaceDiceCalgary
)
from trainer.unet_trainer import UNetTrainerCalgary




it = 0

cfg = {
    'debug': True,
    'log': False,
    'description': f'calgary_unet{it}_augmentednnUNet_test',
    'project': 'MICCAI2023-loose_ends',

    # Data params
    'root': '../../../',
    'data_path': 'data/conp-dataset/projects/calgary-campinas/CC359/Reconstructed/',
    'train_site': 6,
    'augment': True,
    'validation': True,
    'channel_out': 8,

    # Training params
    'batch_size': 32,
    'num_batches_per_epoch': 250,
    'lr': 1e-3,
    'epochs': 250,
    'patience': 4
}


if cfg['log']:
    run = wandb.init(reinit=True, 
                     name=cfg['description'],
                     project=cfg['project'], 
                     config=cfg)
    cfg = wandb.config

description = cfg['description']
print(f"Decsription: {description}")

data_path = cfg['root'] + cfg['data_path']
transforms = Transforms()

train_set = CalgaryCampinasDataset(
    data_path=data_path, 
    site=cfg['train_site'], 
    augment=False, 
    normalize=True, 
    split='train', 
    debug=cfg['debug']
)

train_loader = MultiImageSingleViewDataLoader(
    data=train_set, 
    batch_size=cfg['batch_size'], 
    return_orig=False
)

train_augmentor = transforms.get_transforms('global_transforms')

train_gen = MultiThreadedAugmenter(
    data_loader = train_loader, 
    transform = train_augmentor, 
    num_processes = 4, 
    num_cached_per_queue = 2, 
    seeds=None
)

valid_set = CalgaryCampinasDataset(
    data_path=data_path, 
    site=cfg['train_site'], 
    normalize=True, 
    volume_wise=True,
    split='validation'
)

valid_loader = DataLoader(
    valid_set, 
    batch_size=1,
    shuffle=False, 
    drop_last=False, 
    collate_fn=volume_collate
)


# UNET
seg_model   = UNet2D(
    n_chans_in=1, 
    n_chans_out=1, 
    n_filters_init=cfg['channel_out']
)

criterion   = nn.BCEWithLogitsLoss()
eval_metrics = {
    "Volumetric Dice": DiceScoreCalgary(),
    "Surface Dice": SurfaceDiceCalgary()
}

root = cfg['root']
unet_trainer = UNetTrainerCalgary(
    model=seg_model, 
    criterion=criterion, 
    train_generator=train_gen, 
    valid_loader=valid_loader, 
    root=root, 
    eval_metrics=eval_metrics, 
    lr=cfg['lr'],
    num_batches_per_epoch=cfg['num_batches_per_epoch'],
    n_epochs=cfg['epochs'],
    description=description,
    patience=cfg['patience'],
    log=cfg['log']
)

unet_trainer.fit()
metrics = unet_trainer.eval_all(cfg=cfg)
#np.save(f'{root}results-tmp/results/unet/metrics_{description}', metrics)

Decsription: calgary_unet0_augmentednnUNet_test


  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
import os, sys
import torch
from torch.utils.data import DataLoader 
import wandb
import collections
from batchgenerators.dataloading.multi_threaded_augmenter import MultiThreadedAugmenter

#from nnunet.training.network_training.nnUNetTrainerV2 import nnUNetTrainerV2
sys.path.append('../../')
from dataset import ACDCDataset
from data_utils import (
    Transforms,
    MultiImageSingleViewDataLoader
)
from model.unet import UNet2D
from losses import DiceScoreMMS, CrossEntropyTargetArgmax
from trainer.unet_trainer import UNetTrainerACDC


        


it = 0
                        
### Dataloader
## Initialize trainer to get data loaders with data augmentations from training
#     nnUnet_prefix = '../../../nnUNet/'
#     pkl_file          = nnUnet_prefix + 'data/nnUNet_preprocessed/Task500_ACDC/nnUNetPlansv2.1_plans_2D.pkl'
#     fold              = 0
#     output_folder     = nnUnet_prefix + 'results/nnUnet/nnUNet/2d/Task027_ACDC/nnUNetTrainerV2__nnUNetPlansv2.1/'
#     dataset_directory = nnUnet_prefix + 'data/nnUNet_preprocessed/Task500_ACDC'

#     trainer = nnUNetTrainerV2(pkl_file, 0, output_folder, dataset_directory)
#     trainer.initialize()

#     train_loader = trainer.tr_gen
#     valid_loader = trainer.val_gen
DEBUG = False
transforms = Transforms()

train_set = ACDCDataset(
    data="train",
    debug=DEBUG,
    root = '../../../../'
)
train_loader = MultiImageSingleViewDataLoader(
    data=train_set, 
    batch_size=32, 
    return_orig=False
)    
train_augmentor = transforms.get_transforms('global_transforms')
train_gen = MultiThreadedAugmenter(
    data_loader = train_loader, 
    transform = train_augmentor, 
    num_processes = 4, 
    num_cached_per_queue = 2, 
    seeds=None
)

val_set = ACDCDataset(
    data="val",
    debug=DEBUG,
    root = '../../../../'
)
valid_loader = MultiImageSingleViewDataLoader(
    data=val_set, 
    batch_size=32, 
    return_orig=False
)
valid_augmentor = transforms.get_transforms('io_transforms')
valid_gen = MultiThreadedAugmenter(
    data_loader = valid_loader, 
    transform = valid_augmentor, 
    num_processes = 4, 
    num_cached_per_queue = 2, 
    seeds=None
)

cfg = {
    'debug': False,
    'log': True,
    'description': f'acdc_unet8_{it}',
    'project': 'MICCAI2023',

    # Data params
    'n': 0,
    'root': '../../',
    'validation': True,
    'channel_out': 8,

    # Training params
    'batch_size': 32,
    'lr': 1e-3,
    'epochs': 250,
    'patience': 8
}


if cfg['log']:
    run = wandb.init(reinit=True, 
                     name=cfg['description'],
                     project=cfg['project'], 
                     config=cfg)
    cfg = wandb.config


seg_model    = UNet2D(n_chans_in=1, n_chans_out=4, n_filters_init=cfg['channel_out'])
criterion    = CrossEntropyTargetArgmax()
eval_metrics = {
    "Volumetric Dice": DiceScoreMMS()
} 
root = cfg['root']
unet_trainer = UNetTrainerACDC(model=seg_model,
                               criterion=criterion,
                               train_loader=train_gen,
                               valid_loader=valid_gen,
                               num_batches_per_epoch=250,
                               num_val_batches_per_epoch=50,
                               root=root,
                               eval_metrics=eval_metrics,
                               lr=cfg['lr'],
                               n_epochs=cfg['epochs'],
                               description=cfg["description"],
                               patience=cfg['patience'],
                               log=cfg['log'])

unet_trainer.fit()



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

loading dataset
loading all case properties
loading dataset
loading all case properties


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jlennartz. Use `wandb login --relogin` to force relogin


  0%|          | 0/250 [00:00<?, ?it/s]

ERROR:root:MultiThreadedGenerator: caught exception: (<class 'KeyboardInterrupt'>, KeyboardInterrupt(), <traceback object at 0x7fed4df948c0>)


KeyboardInterrupt: 